In [173]:
import pandas as pd


original = pd.read_csv('data.csv')
original = original.drop(['都道府県名ローマ字', '市区町村名ローマ字', '大字町丁目コード', '大字町丁目名', '緯度', '経度'], axis=1)
original = original.rename(columns={'都道府県コード':  'prefecture_code', 
                                                                 '都道府県名': ' prefecture_name',
                                                                 '都道府県名カナ': 'prefecture_name_ruby', 
                                                                 '市区町村コード': 'city_code',
                                                                 '市区町村名': 'city_name',
                                                                 '市区町村名カナ': 'city_name_ruby',
                                                            })

prefecture = original.iloc[:,:2]
counter_prefecture = prefecture.iloc[0, 0]

city = original.iloc[:,3:]
counter_city = city.iloc[0,1]
drop_list = []
for i in range(1, len(city)):
    if city.iloc[i, 1] == counter:
        drop_list.append(i)
    else:
        counter_city = city.iloc[i, 1]
        continue
        
city.drop(drop_list)


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        prefecture_code  prefecture_name
0                     1              北海道
1                     1              北海道
2                     1              北海道
3                     1              北海道
4                     1              北海道
...                 ...              ...
190011               47              沖縄県
190012               47              沖縄県
190013               47              沖縄県
190014               47              沖縄県
190015               47              沖縄県

[190016 rows x 2 columns]


,city_code,city_name,city_name_ruby
0,1101,札幌市中央区,サッポロシチュウオウク
990,1102,札幌市北区,サッポロシキタク
2047,1103,札幌市東区,サッポロシヒガシク
3152,1104,札幌市白石区,サッポロシシロイシク
3612,1105,札幌市豊平区,サッポロシトヨヒラク
...,...,...,...
189949,47361,島尻郡久米島町,シマジリグンクメジマチョウ
189976,47362,島尻郡八重瀬町,シマジリグンヤエセチョウ
189999,47375,宮古郡多良間村,ミヤコグンタラマソン
190002,47381,八重山郡竹富町,ヤエヤマグンタケトミチョウ
